In [10]:
!pip install --upgrade imbalanced-learn
# !pip install --upgrade scikit-learn
!pip install --upgrade scikit-learn imbalanced-learn

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from causalml.inference.meta import BaseXClassifier
from imblearn.over_sampling import SMOTE
seed = 1

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
Failed to import duecredit due to No module named 'duecredit'


# Dataset

In [2]:
train_set = pd.read_csv(r'/Users/marcus/Documents/ae hackathon/dataset/65d4f0fcb8af9_amex_campus_challenge_train_3.csv')
eval_set = pd.read_csv('/Users/marcus/Documents/ae hackathon/dataset/65d4b8b2ebfe9_amex_campus_challenge_eval_round1_2.csv' )


In [3]:
must_keep_train_cols = ["ind_recommended","activation"]

# Data preprocessing

In [4]:
train_set.fillna(0.0, inplace=True)

In [24]:
# Move 'distance_05' column to the first position for train_set_recommended
if('distance_05' in train_set.columns):
    col = train_set.pop('distance_05')
    train_set.insert(0, col.name, col)

train_set_recommended = train_set[train_set['ind_recommended'] == 1].drop(columns=['customer', 'merchant'],axis=1,inplace=False)
train_set_not_recommended = train_set[train_set['ind_recommended'] == 0].drop(columns=[ 'customer', 'merchant'],axis=1,inplace=False)

In [26]:
def balance_classes_with_smote(df, class_column, sample_size, seed, k_neighbors=5):
    smote = SMOTE(sampling_strategy='auto', k_neighbors=k_neighbors)

    # Sample rows where activation is 0 and where activation is 1
    filtered_non_activation = df[df[class_column] == 0].sample(n=sample_size, random_state=seed)
    filtered_activation = df[df[class_column] == 1]

    # Concatenate the filtered dataframes
    df_balanced = pd.concat([filtered_non_activation, filtered_activation], axis=0)

    # Apply SMOTE to the activation data
    X_resampled, y_resampled = smote.fit_resample(df_balanced.drop([class_column], axis=1), df_balanced[class_column])

    # Combine the resampled data
    df_resampled = pd.DataFrame(X_resampled, columns=df_balanced.drop([class_column], axis=1).columns)
    df_resampled[class_column] = y_resampled

    return df_resampled

def balance_classes(df, class_column, sample_size, seed):
    filtered_non_activation = df[df[class_column] == 0].sample(n=sample_size, random_state=seed)
    filtered_activation = df[df[class_column] == 1].sample(n=sample_size, random_state=seed)

    return pd.concat([filtered_non_activation, filtered_activation], axis=0)


# Apply balance_classes_with_smote function to train_set_recommended
train_set_recommended = balance_classes_with_smote(train_set_recommended, 'activation', 30000, seed)

# Apply balance_classes_with_smote function to train_set_not_recommended
train_set_not_recommended = balance_classes(train_set_not_recommended, 'activation', 30000, seed)

In [28]:
train_total = pd.concat([train_set_recommended,train_set_not_recommended],axis=0).sample(frac=1,random_state=seed)

# Uplift Model

In [34]:
x_learner = BaseXClassifier(outcome_learner=RandomForestClassifier(n_estimators=100,random_state=seed)
                            , effect_learner=RandomForestRegressor(n_estimators=100,random_state=seed))

x_learner.fit(
    train_total.drop(columns=['activation','ind_recommended'],axis=1,inplace=False).values,
    treatment=train_total['ind_recommended'].values,
    y = train_total['activation'].values 
)

In [37]:
y = x_learner.predict(eval_set.fillna(0).drop(columns=['merchant', 'customer'],axis=1,inplace=False).values)
submission = eval_set[["customer","merchant"]].assign(predicted_score=y)

In [38]:
submission.to_csv('submission.csv',index=False)